# Imports and set up

In [148]:
import os, sys
import re

import requests
from lxml import etree
from bs4 import BeautifulSoup

from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio import Entrez

In [12]:
# Set up the Entrez email address
Entrez.email = "timofei.ryko@gmail.com"

# Set up the search parameters
query = "Coronaviridae antigen"
db = "cdd"

# Search protein identifiers by term

In [7]:
# Perform the Entrez search to get the unique identifiers
handle = Entrez.esearch(db=db, term=query)
record = Entrez.read(handle)
handle.close()

# Get the list of unique identifiers
ids = record['IdList']

In [8]:
ids

['465119', '394953', '394951', '394949', '394831', '394823']

# Find conserved domains (FAILED)

In [126]:
DATA = {
    'tdata': 'aligns',
    'alnfmt': 'json',
    'dmode': 'all'
}

In [127]:
QUERY = f"https://www.ncbi.nlm.nih.gov/Structure/bwrpsb/bwrpsb.cgi?queries={'%0A'.join(ids)}&&tdata=aligns&alnfmt=json&dmode=all"
QUERY

'https://www.ncbi.nlm.nih.gov/Structure/bwrpsb/bwrpsb.cgi?queries=465119%0A394953%0A394951%0A394949%0A394831%0A394823&&tdata=aligns&alnfmt=json&dmode=all'

In [128]:
response = requests.get(QUERY, data=DATA)
response

<Response [200]>

In [129]:
search_data = response.json()
search_data

{'Seq_annot': {'id': [{'local': {'str': 'QM3-qcdsearch-C05004FAA30DADE-3243E42F47818BC9'}}],
  'name': 'Batch CD-search tool - NIH/NLM/NCBI',
  'desc': [{'comment': 'status=3'}, {'comment': 'msg=Job is still running'}],
  'data': {'ids': [{'local': {'str': 'aligns Standard Results'}}]}}}

In [130]:
search_id = search_data['Seq_annot']['id'][0]['local']['str']

SEARCH_QUERY = f"https://www.ncbi.nlm.nih.gov/Structure/bwrpsb/bwrpsb.cgi?cdsid={search_id}&tdata=aligns&alnfmt=json&dmode=all"
SEARCH_QUERY

'https://www.ncbi.nlm.nih.gov/Structure/bwrpsb/bwrpsb.cgi?cdsid=QM3-qcdsearch-C05004FAA30DADE-3243E42F47818BC9&tdata=aligns&alnfmt=json&dmode=all'

In [131]:
response = requests.post(SEARCH_QUERY, data=DATA)
response

<Response [200]>

In [132]:
response.json()

{'Seq_annot': {'id': [{'local': {'str': 'QM3-qcdsearch-C05004FAA30DADE-3243E42F47818BC9'}}],
  'name': 'Batch CD-search tool - NIH/NLM/NCBI',
  'desc': [{'comment': 'status=0'}],
  'data': {'ids': [{'local': {'str': 'aligns Standard Results'}}]}}}

# Convert PSSM ID to a CDD (FAILED)

In [110]:
records = []
for id in ids:
    handle = Entrez.efetch(db=db, id=id, rettype="gb", retmode="text")
    record = Entrez.read(handle)
    records.append(record)
    handle.close()


In [111]:
records[0]

['465119']

In [109]:
ids

['465119', '394953', '394951', '394949', '394831', '394823']

# BAD convert of IDs using bs4

In [140]:
page = requests.get('https://www.ncbi.nlm.nih.gov/Structure/cdd/cddsrv.cgi?uid=394953')

In [135]:
soup = BeautifulSoup(page.content, 'html.parser')

In [156]:
div = soup.find('div', class_='lkbct', id='leftmenu-0')
link = div.find('a', class_='alink')
accession = link.text
# Not a good way!

In [165]:
div = soup.find('div', class_='title')
links = div.find_all('a')
for link in links:
    href = link.get('href', '')
    # Use regex to check if href starts with the desired pattern
    if re.match(r'^/Structure/cdd/cddsrv\.cgi\?.*', href):
        matching_link = link
        break  # Stop after finding the first matching link

if matching_link:
    # Extract the href and parse the uid parameter
    href = matching_link['href']
    uid = re.search(r'uid=([^&]+)', href)  # Extract uid using regex

uid.group(1)

'cd21470'

In [149]:
prefixes = [
    "Source Database",
    "cd",
    "sd",
    "NF",
    "pfam",
    "smart",
    "COG",
    "KOG",
    "PRK",
    "CHL",
    "MTH",
    "PHA",
    "PLN",
    "PTZ",
    "TIGR",
    "LOAD_"
]

# Prepare a regex pattern to match prefixes followed by digits
prefix_pattern = r'|'.join([re.escape(prefix) + r'\d+' for prefix in prefixes])

In [166]:
cd_ids = []

for pssm_id in ids:

    page = requests.get(f'https://www.ncbi.nlm.nih.gov/Structure/cdd/cddsrv.cgi?uid={pssm_id}')
    soup = BeautifulSoup(page.content, 'html.parser')

    div = soup.find('div', class_='title')
    links = div.find_all('a')
    for link in links:
        href = link.get('href', '')
        # Use regex to check if href starts with the desired pattern
        if re.match(r'^/Structure/cdd/cddsrv\.cgi\?.*', href):
            matching_link = link
            break  # Stop after finding the first matching link

    if matching_link:
        # Extract the href and parse the uid parameter
        href = matching_link['href']
        uid_match = re.search(r'uid=([^&]+)', href)  # Extract uid using regex
        uid = uid_match.group(1)
        # Check if uid starts with any of the specified prefixes followed by digits
        if re.match(rf'^{prefix_pattern}', uid):
            cd_ids.append(uid)
        else:
            print(f"UID {uid} does not match the required pattern.")

assert len(ids) == len(cd_ids)

In [167]:
cd_ids

['pfam16451', 'cd21627', 'cd21625', 'cd21527', 'cd21484', 'cd21470']

# Get FASTA files

All the FASTA files were downloaded from https://ftp.ncbi.nih.gov/pub/mmdb/cdd, extracted from `fasta.tar.gz` archive

In [168]:
FASTA_DIR_PATH = '/home/timofeiryko/Downloads/fasta'

consensus_sequences = {}

for uid in cd_ids:
    filename = os.path.join(FASTA_DIR_PATH, f'{uid}.FASTA')
    
    # Check if the file exists
    if os.path.exists(filename):
        with open(filename, 'r') as fasta_file:
            # Parse the FASTA file
            for record in SeqIO.parse(fasta_file, 'fasta'):
                # Check if the record id indicates a consensus sequence
                if record.id.startswith('lcl|consensus'):
                    consensus_sequences[uid] = str(record.seq)  # Store the consensus sequence
                    break  # Stop after finding the consensus sequence

In [174]:
for el in consensus_sequences.values():
    print(el)
    print(len(el))
    print('---')

DVSKADG--TYYLPDRVY-SNTTTLRQGLFPKQGDNVTNY-VYSPAH-------HCGKR-------NFSTPF---LP--------FGD-GIFVHIGNASNA---TGGRIIS-----------EPPAFVFGSTFGNTS----------HTLIIAPDS----------CQTNLTILVC-----------------NFTLCANPVTACRWG-AGNYNANNSLVYFK---NAINCTFNRTY-NITFDTS-----------LIYFGFKQQDGGFHIYYSYWL--------PDLDSGPPTLFPFATLPLGINITYFQVIPS---SIR----STQN------CRRANAAYYVAPLKPSTFLLDFDVNGYITNAVDCSFDPLSELKCSTGSFEPDTGVYSTSSYRAQPVGSVYVRQPNVTCD
414
---
HNITYVSFSV-NKVSRLLPDPYIAYSGQTVRQSLYVADTSNTTVYPVTPPAVGG-----KPGIYNTTILPVGDGLFVHTYMYRQQ-------DSSNTYCQEPFGVAFGNTFEQDRIAIVIIAPGNLGSWPAVSKRTTTNVHILVCSNATLCANPAFNRWGPAGSI--YASDAFVCHGNSCFVNNTFII-PINTSRINLAFRFKDGNLLLYHSAWLPTSG---LNLSGDYPLHYYMSVPVGFNLPNAQFFQSVVRPNTEP------ADGACAAFQNNLYIAPLSKRELLVSYDDNGSPVNVADCSADAGSELYCV
314
---
IGDFKCTT-----VSINDVNTGAPSISTETVDVSNGLGTYYVLDRVYLNTTLLLNGYYPTSGSTYRNLALKGTLLLSTLWFKPPFLSEFNNGIFAKVKNTKVSKNG-VMYSEFPTIVIGSTFVNTSYTVVVQPHTGNSDN----KLQGILEISVCQYTMCEYPNTICKPNL-GNQRIELWHTDTGV-PSCLYKRNFTYDVNADWLYFHFYQEGGTFYAYYADTGSVTTFLFSVYLGTVLSHYYVMPLTCNST--

In [175]:
consensus_sequences

{'pfam16451': 'DVSKADG--TYYLPDRVY-SNTTTLRQGLFPKQGDNVTNY-VYSPAH-------HCGKR-------NFSTPF---LP--------FGD-GIFVHIGNASNA---TGGRIIS-----------EPPAFVFGSTFGNTS----------HTLIIAPDS----------CQTNLTILVC-----------------NFTLCANPVTACRWG-AGNYNANNSLVYFK---NAINCTFNRTY-NITFDTS-----------LIYFGFKQQDGGFHIYYSYWL--------PDLDSGPPTLFPFATLPLGINITYFQVIPS---SIR----STQN------CRRANAAYYVAPLKPSTFLLDFDVNGYITNAVDCSFDPLSELKCSTGSFEPDTGVYSTSSYRAQPVGSVYVRQPNVTCD',
 'cd21627': 'HNITYVSFSV-NKVSRLLPDPYIAYSGQTVRQSLYVADTSNTTVYPVTPPAVGG-----KPGIYNTTILPVGDGLFVHTYMYRQQ-------DSSNTYCQEPFGVAFGNTFEQDRIAIVIIAPGNLGSWPAVSKRTTTNVHILVCSNATLCANPAFNRWGPAGSI--YASDAFVCHGNSCFVNNTFII-PINTSRINLAFRFKDGNLLLYHSAWLPTSG---LNLSGDYPLHYYMSVPVGFNLPNAQFFQSVVRPNTEP------ADGACAAFQNNLYIAPLSKRELLVSYDDNGSPVNVADCSADAGSELYCV',
 'cd21625': 'IGDFKCTT-----VSINDVNTGAPSISTETVDVSNGLGTYYVLDRVYLNTTLLLNGYYPTSGSTYRNLALKGTLLLSTLWFKPPFLSEFNNGIFAKVKNTKVSKNG-VMYSEFPTIVIGSTFVNTSYTVVVQPHTGNSDN----KLQGILEISVCQYTMCEYPNTICKPNL-GNQRIELWHTDTGV-PSCLYKRNFTYDVNADWLYFHFYQEGGTFYAYYADTGS

In [176]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO
from Bio import SeqIO

# Create a list to hold SeqRecord objects
records = []

# Create SeqRecord objects for each entry in the dictionary
for uid, sequence in consensus_sequences.items():
    record = SeqRecord(Seq(sequence), id=uid, description="")
    records.append(record)

# Use StringIO to capture the FASTA formatted output
output = StringIO()
SeqIO.write(records, output, "fasta")
fasta_string = output.getvalue()

# Print the resulting FASTA string
print(fasta_string)

>pfam16451
DVSKADG--TYYLPDRVY-SNTTTLRQGLFPKQGDNVTNY-VYSPAH-------HCGKR-
------NFSTPF---LP--------FGD-GIFVHIGNASNA---TGGRIIS---------
--EPPAFVFGSTFGNTS----------HTLIIAPDS----------CQTNLTILVC----
-------------NFTLCANPVTACRWG-AGNYNANNSLVYFK---NAINCTFNRTY-NI
TFDTS-----------LIYFGFKQQDGGFHIYYSYWL--------PDLDSGPPTLFPFAT
LPLGINITYFQVIPS---SIR----STQN------CRRANAAYYVAPLKPSTFLLDFDVN
GYITNAVDCSFDPLSELKCSTGSFEPDTGVYSTSSYRAQPVGSVYVRQPNVTCD
>cd21627
HNITYVSFSV-NKVSRLLPDPYIAYSGQTVRQSLYVADTSNTTVYPVTPPAVGG-----K
PGIYNTTILPVGDGLFVHTYMYRQQ-------DSSNTYCQEPFGVAFGNTFEQDRIAIVI
IAPGNLGSWPAVSKRTTTNVHILVCSNATLCANPAFNRWGPAGSI--YASDAFVCHGNSC
FVNNTFII-PINTSRINLAFRFKDGNLLLYHSAWLPTSG---LNLSGDYPLHYYMSVPVG
FNLPNAQFFQSVVRPNTEP------ADGACAAFQNNLYIAPLSKRELLVSYDDNGSPVNV
ADCSADAGSELYCV
>cd21625
IGDFKCTT-----VSINDVNTGAPSISTETVDVSNGLGTYYVLDRVYLNTTLLLNGYYPT
SGSTYRNLALKGTLLLSTLWFKPPFLSEFNNGIFAKVKNTKVSKNG-VMYSEFPTIVIGS
TFVNTSYTVVVQPHTGNSDN----KLQGILEISVCQYTMCEYPNTICKPNL-GNQRIELW
HTDTGV-PSCLYKRNFTYDVNADWLYFHFYQEGGTFYAYYADTGSVT

In [178]:
consensus_sequences

{'pfam16451': 'DVSKADG--TYYLPDRVY-SNTTTLRQGLFPKQGDNVTNY-VYSPAH-------HCGKR-------NFSTPF---LP--------FGD-GIFVHIGNASNA---TGGRIIS-----------EPPAFVFGSTFGNTS----------HTLIIAPDS----------CQTNLTILVC-----------------NFTLCANPVTACRWG-AGNYNANNSLVYFK---NAINCTFNRTY-NITFDTS-----------LIYFGFKQQDGGFHIYYSYWL--------PDLDSGPPTLFPFATLPLGINITYFQVIPS---SIR----STQN------CRRANAAYYVAPLKPSTFLLDFDVNGYITNAVDCSFDPLSELKCSTGSFEPDTGVYSTSSYRAQPVGSVYVRQPNVTCD',
 'cd21627': 'HNITYVSFSV-NKVSRLLPDPYIAYSGQTVRQSLYVADTSNTTVYPVTPPAVGG-----KPGIYNTTILPVGDGLFVHTYMYRQQ-------DSSNTYCQEPFGVAFGNTFEQDRIAIVIIAPGNLGSWPAVSKRTTTNVHILVCSNATLCANPAFNRWGPAGSI--YASDAFVCHGNSCFVNNTFII-PINTSRINLAFRFKDGNLLLYHSAWLPTSG---LNLSGDYPLHYYMSVPVGFNLPNAQFFQSVVRPNTEP------ADGACAAFQNNLYIAPLSKRELLVSYDDNGSPVNVADCSADAGSELYCV',
 'cd21625': 'IGDFKCTT-----VSINDVNTGAPSISTETVDVSNGLGTYYVLDRVYLNTTLLLNGYYPTSGSTYRNLALKGTLLLSTLWFKPPFLSEFNNGIFAKVKNTKVSKNG-VMYSEFPTIVIGSTFVNTSYTVVVQPHTGNSDN----KLQGILEISVCQYTMCEYPNTICKPNL-GNQRIELWHTDTGV-PSCLYKRNFTYDVNADWLYFHFYQEGGTFYAYYADTGS